In [1]:
import yaml
import data
from data.read_data_temp import select_data,TrainData,file_split
from data.data_split import kfsplit
from data.datasets import prepare_loaders
from utils.wandb_init import get_run,wandb_utils
from utils.train import get_score,train_set
from utils.trivial import get_logger
from model.head import OriginModel
from model.trainer import train,predict
import torch
from transformers import logging


import wandb

import numpy as np
import pandas as pd 
import os
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score,recall_score,f1_score
import gc 
gc.enable()
from utils.trivial import set_seed

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import math
from sklearn.model_selection import train_test_split


In [3]:
yaml_path="./config/query_url.yaml"
CONFIG,Logger,DEVICE=train_set(yaml_path,experimentName=None,upload=True,filename="./test/logs")

set_seed(CONFIG['SEED'])
CONFIG["run_db"]=False
CONFIG["DEBUG_MODEL"]=False


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/v-mxiong/.netrc


In [4]:
file_split(CONFIG,column_name=["Query","label","UserVector"])
        

file split completed


In [6]:
from data.read_data_temp import read_data_file
class DataByFile(object):
    def __init__(self,CONFIG,data_type,column_name=["Query","label"],train_start_epoch=0):
        self.CONFIG=CONFIG
        self.positivepath=os.path.join(CONFIG["INPUT_DIR"],"positivefold")
        self.negativepath=os.path.join(CONFIG["INPUT_DIR"],"negativefold")
        self.data_type=data_type
        self.column_name=column_name
        # deperate the last uncomplete negative file
        self.iter_num=self.get_negative_fold()-1

        positive_file=os.path.join(self.positivepath,data_type)
        self.positive_df=self.get_fold(read_data_file(positive_file,\
            CONFIG["DEBUG_MODEL"],data_num=CONFIG["DATA_NUM"],head=self.column_name),skip_info=False)
        self.start_num=train_start_epoch
        self.now_num=train_start_epoch
        
    def get_data(self,select_num=None):
        return self.iter(select_num)
    def get_negative_fold(self):
        info_file=os.path.join(self.CONFIG["INPUT_DIR"],"info")
        with open(info_file,"r",encoding="UTF-8") as file_obj:
            message=file_obj.readline().strip()
            message=message.split("\t")

        if self.data_type=="train":
            return int(message[0])
        else:
            return int(message[1])
    def get_fold(self,data_df,skip_info=True):

        data_df["label"]=data_df["label"].replace(["0","1"],[0,1])
        data_df=data_df.query("label==1 or label==0")
        data_df["label"]=data_df["label"].astype(int)
        data_df.reset_index(inplace=True,drop=True)
        data_df=kfsplit(data_df,self.CONFIG,skip_info=skip_info)
        return data_df
    def iter(self,select_num):
        if select_num!=None:
            i=select_num%self.iter_num
        else:
            i=(self.now_num)%self.iter_num
            self.now_num+=1
        negative_file=os.path.join(self.negativepath,self.data_type)

        negative_df=read_data_file(negative_file+str(i),\
            self.CONFIG["DEBUG_MODEL"],data_num=self.CONFIG["DATA_NUM"],head=self.column_name)
        negative_df=self.get_fold(negative_df)
        
        i=(i+1)%self.iter_num
        train_df=pd.concat([self.positive_df,negative_df],axis=0)
        train_df=train_df.sample(frac=1).reset_index(drop=True)

        return train_df

In [7]:
trainDataObj=DataByFile(CONFIG,"train",column_name=["Query","label","UserVector"])
trainData=trainDataObj.get_data()
CONFIG["Logger"].info("-"*10+"trainData label's value_counts"+"-"*10)
CONFIG["Logger"].info(trainData["label"].value_counts())
CONFIG["Logger"].info("-"*10+"trainData fold's value_counts"+"-"*10)
CONFIG["Logger"].info(trainData["fold"].value_counts())

----------data_df fold value_counts----------
1.0    97355
3.0    97355
2.0    97355
0.0    97355
4.0    97354
Name: fold, dtype: int64
----------trainData label's value_counts----------
1    486774
0    486774
Name: label, dtype: int64
----------trainData fold's value_counts----------
1.0    194710
3.0    194710
0.0    194710
2.0    194710
4.0    194708
Name: fold, dtype: int64


In [8]:
testDataObj=DataByFile(CONFIG,"test",column_name=["Query","label","UserVector"])
testData=testDataObj.get_data()
gc.collect()
CONFIG["Logger"].info("-"*10+"testData label's value_counts"+"-"*10)
CONFIG["Logger"].info(testData["label"].value_counts())


----------data_df fold value_counts----------
1.0    24399
2.0    24399
0.0    24399
3.0    24398
4.0    24398
Name: fold, dtype: int64
----------testData label's value_counts----------
1    121993
0    121993
Name: label, dtype: int64


In [13]:
from get_embedding import embedding_floop
from data.read_data_temp import read_data_file

def getQureyEmbedding(data_df,is_exsits,DEVICE,embeddingpath,embedding_name="DataEmbeddings"):
    if not is_exsits or not os.path.exists(os.path.join(embeddingpath,embedding_name)):
        embedding_floop(CONFIG,data_df,DEVICE,embedding_path,embedding_name=embedding_name)
    
    # positive * datanum+negative *datanum
    embedding_df=read_data_file(os.path.join(embeddingpath,embedding_name),CONFIG["DEBUG_MODEL"],CONFIG["DATA_NUM"]*2,\
                               head=["Query","Embedding"])
    CONFIG["Logger"].info(f"query df query0:{embedding_df.loc[0,'Query']},url df query0:{data_df.loc[0,'Query']}")
#     print(embedding_df.head(1))
#     print(data_df.head(1))
    
    embeddings=embedding_df["Embedding"].values.tolist()
    del embedding_df
    return embeddings

def getEmbeddingLabel(data_df,is_exists,DEVICE,embeddingpath,embedding_name):
    
    embeddings=getQureyEmbedding(data_df,is_exists,DEVICE,embeddingpath,embedding_name)
    full_embeddings,url_embeddings,query_embeddings=[],[],[]
    drop_index=[]
    for i in tqdm(range(len(embeddings)),ncols=10):

        url_embedding=list(map(lambda x:float(x)/127,data_df.loc[i,"UserVector"].split("|")))
        query_embedding=list(map(lambda x:float(x) ,embeddings[i].split("|")))
        if len(query_embedding)!=768 or len(url_embedding)!=100  :
            drop_index.append(i)
            continue
        full_embeddings.append(url_embedding+query_embedding)
        url_embeddings.append(url_embedding)
        query_embeddings.append(query_embedding)
        
    labels=data_df.query("index not in @drop_index")["label"].values.tolist()
    assert len(url_embeddings)==len(query_embeddings)
    assert len(url_embeddings)==len(full_embeddings)
    assert len(labels)==len(full_embeddings)
    CONFIG["Logger"].info(f"drop num: {len(drop_index)}")
    return full_embeddings,query_embeddings,url_embeddings,labels

embedding_path="./embeddings"




In [15]:
train_embedding_name="TrainEmbedding"

full_train,query_train,url_train,Y_train=getEmbeddingLabel(trainData,True,DEVICE,embedding_path,train_embedding_name)


query df query0:wandern von gertenbach nach burg ludwigstein,url df query0:wandern von gertenbach nach burg ludwigstein
100%|█| 973548/973548 [09:22<00:00, 1731.66it/s] 
drop num: 0


In [14]:
test_embedding_name="TestEmbedding"
full_test,query_test,url_test,Y_test=getEmbeddingLabel(testData,True,DEVICE,embedding_path,test_embedding_name)


query df query0:förderung der stadt nürnberg für meisterschule handwerk,url df query0:förderung der stadt nürnberg für meisterschule handwerk
100%|█| 243986/243986 [02:13<00:00, 1821.25it/s]
drop num: 0


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import classification_report
def linearTrain(X_train,Y_train,X_test,Y_test,C=0.01):
    assert len(X_train)==len(Y_train)
    assert len(X_test)==len(Y_test)
    
    ss=StandardScaler()
    X_train=ss.fit_transform(X_train)
    lg =linear_model.LogisticRegression(C=C)  # 默认使用L2正则化避免过拟合，C=1.0表示正则力度
    lg.fit(X_train, Y_train)
    
    X_test=ss.transform(X_test)
    y_predict = lg.predict(X_test)
    print("训练集准确率：", lg.score(X_train, Y_train))
    print("准确率：", lg.score(X_test, Y_test))
    print("召回率：", classification_report(Y_test, y_predict, labels=[0, 1], target_names=["不满意", "满意"]))

In [20]:
# for full embeddings
linearTrain(full_train,Y_train,full_test,Y_test)

linearTrain(query_train,Y_train,query_test,Y_test)

linearTrain(url_train,Y_train,url_test,Y_test)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


训练集准确率： 0.593688241360467
准确率： 0.591169165443919
召回率：               precision    recall  f1-score   support

         不满意       0.59      0.60      0.59    121993
          满意       0.59      0.59      0.59    121993

    accuracy                           0.59    243986
   macro avg       0.59      0.59      0.59    243986
weighted avg       0.59      0.59      0.59    243986



/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


训练集准确率： 0.5821859836392248
准确率： 0.5804964219258482
召回率：               precision    recall  f1-score   support

         不满意       0.58      0.61      0.59    121993
          满意       0.59      0.55      0.57    121993

    accuracy                           0.58    243986
   macro avg       0.58      0.58      0.58    243986
weighted avg       0.58      0.58      0.58    243986

训练集准确率： 0.5705296503100001
准确率： 0.5709057077045404
召回率：               precision    recall  f1-score   support

         不满意       0.57      0.56      0.57    121993
          满意       0.57      0.58      0.58    121993

    accuracy                           0.57    243986
   macro avg       0.57      0.57      0.57    243986
weighted avg       0.57      0.57      0.57    243986



In [50]:
from torch.utils.data import Dataset, TensorDataset, SequentialSampler, RandomSampler, DataLoader
class DatasetEmbedding(Dataset):
    def __init__(self,query,label,is_test=False):
        self.text=np.asarray(query)
        self.is_test=is_test
        if self.is_test==False:
            self.label=np.asarray(label)
    def __len__(self):
        return len(self.text)
    def __getitem__(self,item):
        text=self.text[item]

        features={"text":torch.tensor(text,dtype=torch.float)}
        if self.is_test==False:
            label=self.label[item]
            features={"text":torch.tensor(text,dtype=torch.float),"label":torch.tensor(label,dtype=torch.long)}
        return features

def make_dataloader(query,label,batch_size,is_test=False):
    dataset=DatasetEmbedding(query,label,is_test=is_test)
    sampler=SequentialSampler(dataset)
    
    dataloader=DataLoader(dataset,
                          batch_size=batch_size,
                          sampler=sampler
                         )
    return dataloader


class Model(torch.nn.Module):
    
    def __init__(self,num_i,num_h,num_o):
        super(Model,self).__init__()
        
        self.linear1=torch.nn.Linear(num_i,num_h)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(num_h,num_h) #2个隐层
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(num_h,num_o)
  
    def forward(self,text,label):
        x = text
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        return x

In [81]:
import time
def trainer(X_train,Y_train,X_test,Y_test,lr=0.01,batch_size=102400,epochs=5):
    
    ss=StandardScaler()
    X_train=ss.fit_transform(X_train)
    X_test=ss.transform(X_test)
    data_loader_train=make_dataloader(X_train,Y_train,batch_size)
    data_loader_test=make_dataloader(X_test,Y_test,batch_size)

    num_i=len(X_train[0])
    num_h,num_o=num_i,2
    model=Model(num_i,num_h,num_o)
    model.to(DEVICE)
    cost = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters())

    for epoch in range(epochs) :
        print(f"----------epoch: {epoch}------------")
        model.train()
        sum_loss=0
        train_correct=0
        for index,batch in enumerate(tqdm(data_loader_train,ncols=10)):

            inputs={key: value.to(DEVICE) for key,value in batch.items()}

            outputs=model(**inputs)
            optimizer.zero_grad()
            loss=cost(outputs,inputs["label"])
            loss.backward()
            optimizer.step()

            _,id=torch.max(outputs.data,1)
            sum_loss+=loss.data
            train_correct+=torch.sum(id==inputs["label"].data)
        print('[%d,%d] loss:%.03f' % (epoch + 1, epochs, sum_loss / len(data_loader_train)))
        print('correct:%.03f%%' % (100 * train_correct / (len(data_loader_train)*batch_size)))
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        ## test 
        model.eval()
        test_correct = 0
        for index,batch in enumerate(tqdm(data_loader_test)) :
            inputs={key: value.to(DEVICE) for key,value in batch.items()}
            outputs=model(**inputs)
            _, id = torch.max(outputs.data, 1)
            test_correct += torch.sum(id == inputs["label"].data)
        print("correct:%.3f%%" % (100 * test_correct / (len(data_loader_test)*batch_size) ))
    del model
    del data_loader_train,data_loader_test
    gc.collect()


In [83]:
 trainer(full_train,Y_train,full_test,Y_test,batch_size=102400,epochs=20)






  0%| | 0/10 [00:00<?, ?it/s]

----------epoch: 0------------







 10%| | 1/10 [00:15<02:17, 15.27s/it]




 20%|▏| 2/10 [00:18<01:32, 11.55s/it]




 30%|▎| 3/10 [00:20<01:02,  8.93s/it]




 40%|▍| 4/10 [00:35<01:04, 10.73s/it]




 50%|▌| 5/10 [00:38<00:41,  8.35s/it]




 60%|▌| 6/10 [00:41<00:26,  6.68s/it]




 70%|▋| 7/10 [00:44<00:16,  5.52s/it]




 80%|▊| 8/10 [00:59<00:16,  8.32s/it]




 90%|▉| 9/10 [01:01<00:06,  6.66s/it]




100%|█| 10/10 [01:03<00:00,  5.09s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[1,20] loss:0.728
correct:50.296%
2022-08-05 04:58:50







 33%|███▎      | 1/3 [00:02<00:05,  2.78s/it]




 67%|██████▋   | 2/3 [00:17<00:06,  6.24s/it]




100%|██████████| 3/3 [00:18<00:00,  4.69s/it]




  0%| | 0/10 [00:00<?, ?it/s]

...

correct:48.128%
----------epoch: 19------------







 10%| | 1/10 [00:13<02:02, 13.63s/it]




 20%|▏| 2/10 [00:16<01:23, 10.38s/it]




 30%|▎| 3/10 [00:19<00:56,  8.10s/it]




 40%|▍| 4/10 [00:32<00:58,  9.75s/it]




 50%|▌| 5/10 [00:35<00:38,  7.67s/it]




 60%|▌| 6/10 [00:38<00:24,  6.20s/it]




 70%|▋| 7/10 [00:52<00:25,  8.60s/it]




 80%|▊| 8/10 [00:55<00:13,  6.86s/it]




 90%|▉| 9/10 [00:58<00:05,  5.64s/it]




100%|█| 10/10 [00:59<00:00,  4.37s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[20,20] loss:0.637
correct:60.265%
2022-08-05 05:21:52







 33%|███▎      | 1/3 [00:14<00:29, 14.91s/it]




 67%|██████▋   | 2/3 [00:17<00:11, 11.27s/it]




100%|██████████| 3/3 [00:18<00:00,  8.21s/it]

correct:48.016%


In [84]:
 trainer(query_train,Y_train,query_test,Y_test,batch_size=102400,epochs=20)






  0%| | 0/10 [00:00<?, ?it/s]

----------epoch: 0------------







 10%| | 1/10 [00:02<00:26,  2.93s/it]




 20%|▏| 2/10 [00:05<00:23,  2.90s/it]




 30%|▎| 3/10 [00:08<00:20,  2.86s/it]




 40%|▍| 4/10 [00:23<00:39,  6.53s/it]




 50%|▌| 5/10 [00:26<00:27,  5.41s/it]




 60%|▌| 6/10 [00:29<00:18,  4.62s/it]




 70%|▋| 7/10 [00:43<00:22,  7.66s/it]




 80%|▊| 8/10 [00:46<00:12,  6.20s/it]




 90%|▉| 9/10 [00:49<00:05,  5.17s/it]




100%|█| 10/10 [00:50<00:00,  4.04s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[1,20] loss:0.705
correct:50.326%
2022-08-05 05:24:59







 33%|███▎      | 1/3 [00:14<00:29, 14.86s/it]




 67%|██████▋   | 2/3 [00:17<00:11, 11.24s/it]




100%|██████████| 3/3 [00:18<00:00,  8.19s/it]




  0%| | 0/10 [00:00<?, ?it/s]

...

correct:46.462%
----------epoch: 19------------







 10%| | 1/10 [00:02<00:25,  2.80s/it]




 20%|▏| 2/10 [00:05<00:22,  2.80s/it]




 30%|▎| 3/10 [00:19<00:43,  6.14s/it]




 40%|▍| 4/10 [00:22<00:30,  5.14s/it]




 50%|▌| 5/10 [00:25<00:22,  4.43s/it]




 60%|▌| 6/10 [00:27<00:15,  3.94s/it]




 70%|▋| 7/10 [00:41<00:20,  6.84s/it]




 80%|▊| 8/10 [00:44<00:11,  5.62s/it]




 90%|▉| 9/10 [00:47<00:04,  4.76s/it]




100%|█| 10/10 [00:48<00:00,  3.75s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[20,20] loss:0.671
correct:56.281%
2022-08-05 05:48:00







 33%|███▎      | 1/3 [00:13<00:27, 13.69s/it]




 67%|██████▋   | 2/3 [00:16<00:10, 10.42s/it]




100%|██████████| 3/3 [00:17<00:00,  7.61s/it]

correct:44.896%


In [85]:
 trainer(url_train,Y_train,url_test,Y_test,batch_size=102400,epochs=20)






  0%| | 0/10 [00:00<?, ?it/s]

----------epoch: 0------------







 10%| | 1/10 [00:02<00:23,  2.66s/it]




 20%|▏| 2/10 [00:05<00:21,  2.67s/it]




 30%|▎| 3/10 [00:08<00:18,  2.68s/it]




 40%|▍| 4/10 [00:23<00:39,  6.50s/it]




 50%|▌| 5/10 [00:25<00:26,  5.31s/it]




 60%|▌| 6/10 [00:28<00:18,  4.52s/it]




 70%|▋| 7/10 [00:42<00:22,  7.36s/it]




 80%|▊| 8/10 [00:45<00:11,  5.96s/it]




 90%|▉| 9/10 [00:47<00:04,  4.93s/it]




100%|█| 10/10 [00:49<00:00,  3.87s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[1,20] loss:0.690
correct:50.152%
2022-08-05 05:49:37







 33%|███▎      | 1/3 [00:14<00:29, 14.75s/it]




 67%|██████▋   | 2/3 [00:17<00:11, 11.08s/it]




100%|██████████| 3/3 [00:18<00:00,  8.07s/it]




  0%| | 0/10 [00:00<?, ?it/s]

...

correct:47.105%
----------epoch: 19------------







 10%| | 1/10 [00:02<00:22,  2.51s/it]




 20%|▏| 2/10 [00:05<00:20,  2.51s/it]




 30%|▎| 3/10 [00:18<00:41,  5.93s/it]




 40%|▍| 4/10 [00:21<00:29,  4.91s/it]




 50%|▌| 5/10 [00:23<00:20,  4.18s/it]




 60%|▌| 6/10 [00:26<00:14,  3.67s/it]




 70%|▋| 7/10 [00:40<00:20,  6.85s/it]




 80%|▊| 8/10 [00:43<00:11,  5.54s/it]




 90%|▉| 9/10 [00:45<00:04,  4.62s/it]




100%|█| 10/10 [00:46<00:00,  3.61s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

[20,20] loss:0.659
correct:57.436%
2022-08-05 06:11:31







 33%|███▎      | 1/3 [00:14<00:28, 14.21s/it]




 67%|██████▋   | 2/3 [00:16<00:10, 10.69s/it]




100%|██████████| 3/3 [00:17<00:00,  7.76s/it]

correct:47.093%


In [18]:
# from sklearn.tree import DecisionTreeClassifier
# import matplotlib.pyplot as plt
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics

# param = {'criterion':['gini'],'max_depth':[30,50,60,100],'min_samples_leaf':[2,3,5,10],'min_impurity_decrease':[0.1,0.2,0.5]}
# grid = GridSearchCV(DecisionTreeClassifier(criterion="entropy"),param_grid=param,cv=6)
# grid.fit(X_train,Y_train)
# print('最优分类器:',grid.best_params_,'最优分数:', grid.best_score_)  # 得到最优的参数和分值

最优分类器: {'criterion': 'gini', 'max_depth': 30, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 2} 最优分数: 0.5009484815244795
